<a href="https://colab.research.google.com/github/nestoredduardo/two-year-churn-predictive-model/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Scientist Hackathon**

# 1. Problem

# 2. Obtain Data

In [1]:
import pandas as pd

In [2]:
tb_clients_path = '/content/drive/MyDrive/Jobs/KIN/clients_table.txt'
tb_credit_score_path = '/content/drive/MyDrive/Jobs/KIN/credit_score_table.txt'
tb_products_path = '/content/drive/MyDrive/Jobs/KIN/products_table.txt'
tb_transactions_path = '/content/drive/MyDrive/Jobs/KIN/transactions_table.txt'

In [3]:
df_clients = pd.read_csv(tb_clients_path)
df_credit_score = pd.read_csv(tb_credit_score_path)
df_products = pd.read_csv(tb_products_path)
df_transactions = pd.read_csv(tb_transactions_path)

# 3. Scrub Data

In [4]:
df_clients.head()

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date
0,15745584,EIRLS,Germany,Female,0.0,1.0,0.00,2018-12-14,NaN,1997-09-18
1,14990118,MOLOCK,Italy,Male,1.0,0.0,121219.28,2019-01-08,NaN,1980-08-03
2,14733224,PAWLUCH,Italy,Female,1.0,1.0,159663.59,2012-08-01,2013-08-09,1977-08-19
3,14648573,NALLS,Spain,Male,1.0,0.0,140827.98,2019-06-19,NaN,1979-02-27
4,15365443,EBERLE,Italy,Male,1.0,0.0,35521.28,2014-01-26,2015-12-04,1972-12-21


In [5]:
df_clients.shape

(1545000, 10)

## Desired Population:
a. Contracts from 2015 onwards.

b. Operations in Italy were closed this year (2019).


c. Take out clients with more than 75% of their info missing.

d. Each client should only have one contract in the database, no more.

e. Remember that for a client to be eligible, it must have at least two years of information within the company (even if it canceled the product).

### a. Contracts from 2015 onwards.

In [6]:
df_clients_desired = df_clients[df_clients['application_date'] >= '2015-01-01']
print(df_clients_desired.head())
print(df_clients_desired.shape)

   CustomerId   Surname Geography  ... application_date   exit_date  birth_date
0    15745584     EIRLS   Germany  ...       2018-12-14         NaN  1997-09-18
1    14990118    MOLOCK     Italy  ...       2019-01-08         NaN  1980-08-03
3    14648573     NALLS     Spain  ...       2019-06-19         NaN  1979-02-27
5    15638124  BRASHERS     Italy  ...       2018-02-23         NaN  1983-01-13
7    14523468  LASKOSKI     Spain  ...       2017-12-28  2018-11-19  1972-10-30

[5 rows x 10 columns]
(623242, 10)


### b. Operations in Italy were closed this year (2019).

In [7]:
df_clients_desired = df_clients_desired[df_clients_desired['Geography'] != 'Italy']
print(df_clients_desired.head())
print(df_clients_desired.shape)

    CustomerId   Surname Geography  ... application_date   exit_date  birth_date
0     15745584     EIRLS   Germany  ...       2018-12-14         NaN  1997-09-18
3     14648573     NALLS     Spain  ...       2019-06-19         NaN  1979-02-27
7     14523468  LASKOSKI     Spain  ...       2017-12-28  2018-11-19  1972-10-30
14    15165393  LABIANCA     Spain  ...       2018-02-22  2019-06-11  1974-07-11
15    14611239     DOKKA    France  ...       2019-02-24         NaN  1986-04-26

[5 rows x 10 columns]
(487424, 10)


### c. Take out clients with more than 75% of their info missing.

In [8]:
df_clients_desired.isna().sum()

CustomerId               0
Surname              23362
Geography            23349
Gender               23349
HasCrCard            23349
IsActiveMember       23349
EstimatedSalary      23349
application_date         0
exit_date           382626
birth_date           23349
dtype: int64

In [9]:
number_info_missing = 75/100 * 9
print(number_info_missing)

6.75


In [10]:
df_clients_desired = df_clients_desired[df_clients_desired.isnull().sum(axis=1) < number_info_missing]
print(df_clients_desired.head())
print(df_clients_desired.shape)

    CustomerId   Surname Geography  ... application_date   exit_date  birth_date
0     15745584     EIRLS   Germany  ...       2018-12-14         NaN  1997-09-18
3     14648573     NALLS     Spain  ...       2019-06-19         NaN  1979-02-27
7     14523468  LASKOSKI     Spain  ...       2017-12-28  2018-11-19  1972-10-30
14    15165393  LABIANCA     Spain  ...       2018-02-22  2019-06-11  1974-07-11
15    14611239     DOKKA    France  ...       2019-02-24         NaN  1986-04-26

[5 rows x 10 columns]
(464075, 10)


### d. Each client should only have one contract in the database, no more.

In [11]:
df_clients_desired.drop_duplicates(subset=['CustomerId'],keep= 'first', inplace= True)
print(df_clients_desired.head())
print(df_clients_desired.shape)

    CustomerId   Surname Geography  ... application_date   exit_date  birth_date
0     15745584     EIRLS   Germany  ...       2018-12-14         NaN  1997-09-18
3     14648573     NALLS     Spain  ...       2019-06-19         NaN  1979-02-27
7     14523468  LASKOSKI     Spain  ...       2017-12-28  2018-11-19  1972-10-30
14    15165393  LABIANCA     Spain  ...       2018-02-22  2019-06-11  1974-07-11
15    14611239     DOKKA    France  ...       2019-02-24         NaN  1986-04-26

[5 rows x 10 columns]
(450562, 10)


### e. Remember that for a client to be eligible, it must have at least two years of information within the company (even if it canceled the product).

In [13]:
today_date = '30/11/2019'

In [21]:
application_date = pd.to_datetime(df_clients_desired['application_date'])
exit_date = pd.to_datetime(df_clients_desired['exit_date'])

In [22]:
df_clients_desired['application_date'] = application_date
df_clients_desired['exit_date'] = exit_date
df_clients_desired.head()

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date
0,15745584,EIRLS,Germany,Female,0.0,1.0,0.00,2018-12-14,NaT,1997-09-18
3,14648573,NALLS,Spain,Male,1.0,0.0,140827.98,2019-06-19,NaT,1979-02-27
7,14523468,LASKOSKI,Spain,Female,1.0,0.0,158161.23,2017-12-28,2018-11-19,1972-10-30
14,15165393,LABIANCA,Spain,Male,1.0,1.0,2612.65,2018-02-22,2019-06-11,1974-07-11
15,14611239,DOKKA,France,Male,0.0,1.0,72210.60,2019-02-24,NaT,1986-04-26


In [27]:
df_clients_desired['exit_date'] - df_clients_desired['application_date']

0              NaT
3              NaT
7         326 days
14        474 days
15             NaT
            ...   
1544983        NaT
1544991        NaT
1544993        NaT
1544996        NaT
1544999        NaT
Length: 450562, dtype: timedelta64[ns]

# 4. Explore Data

# 5. Model Data

# 6. Interpreting Data